<a href="https://colab.research.google.com/github/aknip/Streamlit-Gradio/blob/main/helper_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# helper_lib

In [2]:
!pip install pypandoc==1.11

In [3]:
import pypandoc

def test_pypandoc():
  output = pypandoc.convert_text('# some title', 'rst', format='md')
  return output

#print(test_pypandoc())

In [ ]:
# v4 - 13.08.2023
import shutil
import os
import textwrap
import re
import time
from datetime import datetime
import pypandoc
import os
import zipfile
import random
import requests

def import_prompt_lib():
  fname = 'prompt_lib.ipynb'
  url = 'https://raw.githubusercontent.com/aknip/Streamlit-Gradio/main/' + fname
  r = requests.get(url)
  open(fname, 'wb').write(r.content)
  %run prompt_lib.ipynb

def import_helper_lib():
  fname = 'helper_lib.ipynb'
  url = 'https://raw.githubusercontent.com/aknip/Streamlit-Gradio/main/' + fname
  r = requests.get(url)
  open(fname, 'wb').write(r.content)
  %run helper_lib.ipynb

def create_file_directory(directory, always_delete=False):
  # Creates a new directory - if it not exists yet. The always_delete flag forces a deletion even if it exists.
  # Examples:
  # - create_file_directory('texts', False) => creates a new directory only if it not exists yet
  # - create_file_directory('texts', True) => always deletes existing directory and creates a new one
  if os.path.exists(directory):
    if always_delete:
      # delete the diectory recursively
      shutil.rmtree(directory)
  # create directory
  if not os.path.exists(directory):
    os.mkdir(directory)


def find_files(path, extensions=[".txt"], recursive=False):
    # Recursively (optional) find all files with extension in path
    my_files = []
    for root, dirs, files in os.walk(path):
        for f in files:
            if extensions == []:
                my_files.append(os.path.join(root, f))
            else:
                for ext in extensions:
                    if f.endswith(ext):
                        my_files.append(os.path.join(root, f))
        # no recursion / don't look inside any subdirectory
        if recursive == False:
            break
    return my_files


def merge_textfiles(path, extensions=[".txt"], recursive=False, new_filename='merged.txt'):
    # Recursively (optional) find all files with extension in path
    my_files = find_files(path, extensions, recursive)
    merged_text = ''
    for filename in my_files:
      # print(filename)
      f= open(filename,'r')
      if f.mode == 'r':
            contents =f.read()
      f.close()
      merged_text = merged_text + contents + '\n\n\n'

    f= open(new_filename,'w+')
    f.write(merged_text)
    f.close()


def save_and_backup(filepath, filename, content, convert_to_docx=False):
  dot_pos = filename.rfind('.')
  docx_filename = filename[:dot_pos] + '.docx'
  if os.path.exists(filepath + '/' + filename):
    # file exists, do backup
    current_date_time = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    backuppath = filepath + '/backup ' + current_date_time
    os.mkdir(backuppath)
    shutil.copyfile(filepath + '/' + filename, backuppath + '/' + filename)
    # copy corresponding .docx if necessary
    if (convert_to_docx == True) and (os.path.exists(filepath + '/' + docx_filename)):
      shutil.copyfile(filepath + '/' + docx_filename, backuppath + '/' + docx_filename)
  # save
  f= open(filepath + '/' + filename,'w+')
  f.write(content)
  f.close()
  if convert_to_docx == True:
    docx_file = pypandoc.convert_file(
    filepath + '/' + filename,
    'docx',
    outputfile=filepath + '/' + docx_filename)


def zip_full_archive():
  # zips all project folders (except "sample_data" and config files)
  #os.remove('zipfile.zip')
  directory = "./"
  with zipfile.ZipFile("zipfile.zip", "w") as zip:
      for subdir, dirs, files in os.walk(directory):
          for file in files:
              srcpath = os.path.join(subdir, file)
              if (subdir != './') and (subdir.startswith('./.config') == False) and (subdir != './sample_data'):
                #print(subdir)
                dstpath_in_zip = os.path.relpath(srcpath, start=directory)
                with open(srcpath, 'rb') as infile:
                    print(srcpath)
                    zip.writestr(dstpath_in_zip, infile.read())

def unzip_full_archive():
  with zipfile.ZipFile('zipfile.zip', 'r') as zip:
      zip.printdir()
      zip.extractall('./')


def remove_all_double_whitespace (input_text):
    # replaces ALL whitespace to a single SPACE, no paragraphs left
    clean_text = " ".join(re.split("[\s ]+", input_text, flags=re.UNICODE))
    # clean_text = input_text.translate(str.maketrans('', '', ' \n\t\r'))
    return clean_text.strip()

def remove_double_spaces_keep_paragraphs (input_text):
    # removes double spaces and double paragraphs, keeps paragraph structure
    clean_text = ""
    for line in input_text.split('\n'):
        line_trimmed = line.strip()
        if line_trimmed != '':
            clean_text = clean_text + (re.sub('[\t ]+',' ', line_trimmed)) + '\n'
    return clean_text[:-1] # remove last \n


def remove_special_chars(input_text, strength_level):
  # removes
  cleaned_text = input_text.replace('»', '"')
  cleaned_text = cleaned_text.replace('«', '"')
  cleaned_text = cleaned_text.replace('„', '"')
  cleaned_text = cleaned_text.replace('“', '"')
  cleaned_text = re.sub('(?sm)[\\\/"\'`´\^\*{}\[\]<>\+\&]', '', cleaned_text)
  return cleaned_text


def text_stats(input_text):
    nr_paragraphs = len(input_text.split("\n"))
    nr_words = wordcount(input_text)
    array_segments = input_text.split("§§§")
    nr_segments = len(array_segments)
    stat_txt = f'Number of words / paragraphs / §§§ segements: {nr_words} / {nr_paragraphs} / {nr_segments}'
    if nr_segments > 1:
        seg_stats = []
        seg_stats_text = ''
        for seg in array_segments:
            seg_stats.append(wordcount(seg))
            seg_stats_text = seg_stats_text + str(wordcount(seg)) + ' / '
        stat_txt = stat_txt + '\n\nMax words in §§§ segment: ' + str(max(seg_stats)) + ' (' + seg_stats_text[:-3] + ')'
    return stat_txt



def calculate_stats(my_stats):
  model = my_stats[0]['model_name']
  wordcount_input = 0
  wordcount_output = 0
  completion_tokens = 0
  prompt_tokens = 0
  total_tokens = 0
  for i in my_stats:
    wordcount_input = wordcount_input + i['wordcount']['input']
    wordcount_output = wordcount_output + i['wordcount']['output']
    prompt_tokens = prompt_tokens + i['token_usage']['prompt_tokens']
    completion_tokens = completion_tokens + i['token_usage']['completion_tokens']
    total_tokens = total_tokens + i['token_usage']['total_tokens']

  token_prices = {
    "gpt-3.5-turbo": {
        "prompt_tokens": 0.002,
        "completion_tokens": 0.002,
    },
    "gpt-4": {
        "prompt_tokens": 0.03,
        "completion_tokens": 0.06
    }
  }

  prompt_tokens_price = int((prompt_tokens / 1000 * token_prices[model]["prompt_tokens"] + completion_tokens / 1000 * token_prices[model]["completion_tokens"])*100+0.5)/100
  wordcount_ratio = int(wordcount_output/wordcount_input*100)/100

  stats_obj = {
    'wordcount_input': wordcount_input,
    'wordcount_output': wordcount_output,
    'wordcount_ratio': wordcount_ratio,
    'prompt_tokens': prompt_tokens,
    'completion_tokens': completion_tokens,
    'total_tokens': total_tokens,
    'prompt_tokens_price': prompt_tokens_price
  }

  return stats_obj


def wordcount(input_text):
    # returns number of words of given input_text
    return len(input_text.split())


def split_text_by_separator (text, separator, joiner, fixer, min_words=1000, max_words=0):
    # Split text into an array of texts with a maximum word count
    # optionally use random word counts between min_words and max_words (if max_words is set)
    # example 1: Split by paragraphs
    #   split_text_by_separator(input_text, '\n', '\n', 'fix-nothing', 2000)
    # example 2: Split by sentences '. ' and fix
    #   split_text_by_separator(input_text, '. ', ' ', 'fix-end', 2000)
    # example 3: Split by markdown headlines '\n#' and fix
    #   split_text_by_separator(input_text, '\n#', '\n\n', 'fix-start', 2000)
    paragraphs = []
    sections = []
    section = ''
    if max_words == 0:
      max_words = min_words
    # Split text, separated by separator
    paragraphs_stripped = text.split(separator)
    # fix paragraphs (if wanted) by re-adding the separator at end or start
    if fixer == 'fix-nothing':
        paragraphs = paragraphs_stripped
    if fixer == 'fix-end':
        for index, paragraph in enumerate(paragraphs_stripped):
            if (index+1) == len(paragraphs_stripped):
                paragraphs.append(paragraph)
            else:
                paragraphs.append(paragraph + separator)
    if fixer == 'fix-start':
        for index, paragraph in enumerate(paragraphs_stripped):
            if index == 0:
                paragraphs.append(paragraph)
            else:
                paragraphs.append(separator + paragraph)
    # Loop through paragraphs and aggregate up to maximum word count
    for index, paragraph in enumerate(paragraphs):
        if min_words == max_words:
            max_random = min_words
        else:
            max_random = random.randrange(min_words, max_words)
        test_section = section + paragraph + joiner
        if wordcount(test_section) > max_random:
            sections.append(section.strip())
            section = paragraph
            if min_words == max_words:
                max_random = min_words
            else:
                max_random = random.randrange(min_words, max_words)
        else:
            section = section + paragraph + joiner
            # if last paragraph, append to array
            if (index+1) == len(paragraphs):
              sections.append(section)
    return sections


In [ ]:

import pickle



def write_prompt_to_lib(promptlib, prompt):
    # adds or updates prompt to library
    #
    # TODO: Add model sections "gpt-4", ... etc.
    #
    id = prompt['id']
    version = prompt['version']
    if id not in promptlib:
        # new id
        promptlib[id] = {}
        promptlib[id][version] = {}
    else:
        # id already existing, checking for version
        if version not in promptlib[id]:
            promptlib[id][version] = {}
    promptlib[id]['description'] = prompt['description']
    promptlib[id]['category'] = prompt['category']
    promptlib[id][version]['note'] = prompt['note']
    promptlib[id][version]['prompt'] = prompt['prompt']


def get_prompt_from_lib(promptlib, id, model='default-model', version=None ):
    # searches for prompt with given id
    # looks for highest available version, if no version is given
    if id==None:
        prompt_found = None
    else:
        prompt = promptlib[id]
        if version != None:
            # version given
            version_highest = int(version)
        else:
            # look for highest available version
            version_highest = 0
            for key, val in prompt.items():
                try:
                    version = int(key)
                except ValueError:
                    version = 0
                if version > version_highest:
                    version_highest = version
        prompt_version = prompt[str(version_highest)]
        # return dict
        prompt_found = {}
        prompt_found['id'] = id
        prompt_found['description'] = prompt['description']
        prompt_found['category'] = prompt['category']
        prompt_found['version'] = str(version_highest)
        prompt_found['note'] = prompt_version['note']
        prompt_found['prompt'] = prompt_version['prompt']
        prompt_found['lang-de'] = prompt_version['lang-de']
        prompt_found['lang-en'] = prompt_version['lang-en']
        prompt_found['lang-same'] = prompt_version['lang-same']
        if model in prompt_version:
          prompt_found['input-text-max'] = prompt_version[model]['input-text-max']
          prompt_found['length-max-fix'] = prompt_version[model]['length-max-fix']
          prompt_found['length-max-dyn'] = prompt_version[model]['length-max-dyn']
        else:
          prompt_found['input-text-max'] = prompt_version['default-model']['input-text-max']
          prompt_found['length-max-fix'] = prompt_version['default-model']['length-max-fix']
          prompt_found['length-max-dyn'] = prompt_version['default-model']['length-max-dyn']
    return prompt_found

def build_prompt_from_template(input_txt, prompt_obj, lang, length_type, length_max, toc_text):
  prompt_text =  prompt_obj['prompt']
  language_text = prompt_obj[lang]
  # calculate max. output lengt dynamically
  if length_type != '':
    length_template = prompt_obj[length_type]
    input_txt2 = re.sub('(?sm)[\\\/"\'„“]', '', input_txt)
    input_txt2 = re.sub('(?sm)[\n]', ' ', input_txt2)
    length_template2 = length_template.format(input_text = input_txt2, max_len=length_max)
    length_text = eval(f"f'{length_template2}'")
  else:
    length_text = ''
  # Create the final prompt, fill the variables
  # Example: "Summarize the following text. {language_text} {length_text} Text:```{input_text}``` TOC:```{toc_text}```"
  final_prompt = prompt_text.format(language_text=language_text, length_text=length_text, input_text=input_txt, toc_text=toc_text)
  return final_prompt

def length_calc(my_text, maxWords):
  # maxWords 0.0 - 1.0 (percentage)
    return int(wordcount(my_text) * maxWords)

def save_promptlib(promptlib):
  # saves promptlib to disk
  with open('promptlib.dictionary', 'wb') as dict_file:
      pickle.dump(promptlib, dict_file)

def load_promptlib():
  # load promptlib from disk
  with open('promptlib.dictionary', 'rb') as dict_file:
      promptlib = pickle.load(dict_file)
  return promptlib



In [ ]:
# Talk to model

from datetime import datetime
import langchain
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.schema import HumanMessage
from langchain.callbacks.base import BaseCallbackHandler
import pypandoc

# OpenAI Key
openAI_key = creds['OpenAI']['v2']['credential']

def create_test_project(proj_name, proj_text):
  create_file_directory(proj_name, False)
  create_file_directory(proj_name + '/' +  folders['text-input'], False)
  create_file_directory(proj_name + '/' +  folders['text-output'], False)
  create_file_directory(proj_name + '/' +  folders['text-output-logs'], False)
  if proj_text != '':
    f= open(proj_name + '/' +  folders['text-input'] + '/testfile.txt','w+')
    f.write(proj_text)
    f.close()


def execute_prompt (exec_params):
  #    'model':'no',
  #    'prompt-name': prompt_name,
  #    'prompt-version': prompt_version,
  #    'prompt-comment': prompt_comment,
  #    'loglevel':0,
  #    'stop-after-step':5,
  #    'proj-name': proj_name,
  #    'input-filepath': '',
  #    'input-filename': '',
  #    'folders': folders,
  #    'prompt-obj': prompt_obj,
  #    'language': 'lang-de',
  #    'length-type': 'length-max-dyn',
  #    'length-max': 0.5,
  #    'toc-text': 'some toc-text',
  #    'result-filename': 'paraphrased.txt'

  # Read file from disk
  if exec_params['input-filename'] != '':
    input_file_path = exec_params['proj-name'] + '/' + exec_params['input-filepath'] + '/' + exec_params['input-filename']
  else:
    input_file_path = find_files(exec_params['proj-name'] + '/' + exec_params['folders']['text-input'], ['.txt','.md'], False)[0]
  f= open(input_file_path,'r')
  if f.mode == 'r': input_text =f.read()
  f.close()
  # if input text is output from a prevoius step, strip out Logs
  input_text = input_text.split('=== LOGS: ===')[0]

  # remove special characters
  # input_text = remove_special_chars(input_text, 1) # level 1

  # email.[^[\[1\]]{.underline}^](\l)

  if exec_params['loglevel'] >= 1:
    print(json.dumps(exec_params['prompt-obj'], sort_keys=False, indent=2) + '\n')
    #print(textwrap.fill(input_text, 120) + '\n')

  llm = ChatOpenAI(model_name=exec_params['model'], temperature=0.0, openai_api_key=openAI_key)
  # model overview see https://gptforwork.com/guides/openai-gpt3-models

  #input_text_segments = input_text.split("§§§")
  input_text_segments = split_text_by_separator(input_text, '\n', '\n', 'fix-nothing', exec_params['prompt-obj']['input-text-max'])
  output_text_segments = []

  log_detail_model = 'Model: ' + exec_params['model']
  print(log_detail_model)
  log_time_start = 'Started at: ' + datetime.now().strftime("%H:%M:%S") + '\n'
  print(log_time_start)
  timer_start = time.time()

  log_stats = []
  log_prompt = '\n\n\n***********************\n\n# Promptname: ' + exec_params['prompt-name'] + ', Version: ' + str(exec_params['prompt-version'])  + '\n'
  log_prompt = '\n' + log_prompt + exec_params['prompt-comment'] + '\n\n\n'
  prompt_obj_for_log = json.dumps(exec_params['prompt-obj'], sort_keys=False, indent=2).replace('\n', '<br>')
  prompt_obj_for_log = prompt_obj_for_log.replace(' ', '&nbsp;')
  prompt_obj_for_log = prompt_obj_for_log.replace('`', '\`')
  log_prompt = log_prompt +  prompt_obj_for_log
  log_input_output_compare = '\n\n\n***********************\n\n# Compare input and output texts (' + log_detail_model + ')\n'
  log_input_SHORT = '\n\n\n***********************\n\n# Prompts (Input Text shortened)\n'
  log_input = '\n\n\n***********************\n\n# Prompts (Full length)\n'
  log_input_output = '\n\n\n***********************\n\n# Prompts and anwers (Full length)\n'

  for index, input_text in enumerate(input_text_segments):

      log_detail = 'Processing step ' + str(index + 1) + ' of ' + str(len(input_text_segments))
      print(log_detail)

      #print('TEXT FROM ARRAY:')
      #print(textwrap.fill(input_text, 120) + '\n')

      prompt_txt_final = remove_all_double_whitespace(
          build_prompt_from_template(input_text, exec_params['prompt-obj'], exec_params['language'], exec_params['length-type'], exec_params['length-max'], exec_params['toc-text'])
          )
      # Shorten input text for logging purposes
      #input_txt_short = re.findall('(?sm)```(.+)```', prompt_txt_final)[0][:30]
      #prompt_txt_final_SHORT = re.sub('(```)(.+)(```)', r'\1' + input_txt_short + '...' + r'\3' , prompt_txt_final)
      prompt_txt_final_SHORT = prompt_txt_final[:800]
      if exec_params['loglevel'] <= 1:
        print(textwrap.fill(prompt_txt_final_SHORT, 120) + '\n')
      if exec_params['loglevel'] == 2:
        print(textwrap.fill(prompt_txt_final, 120) + '\n')

      if exec_params['model'] != 'no':
        gpt_response_obj = llm.generate([[HumanMessage(content=prompt_txt_final)]])
        tmp_text = gpt_response_obj.generations[0][0].text
        tmp_tokens = gpt_response_obj.llm_output
        # log statistics (tokens, words)
        tmp_tokens['wordcount'] = {}
        tmp_tokens['wordcount']['input'] = wordcount(input_text)
        tmp_tokens['wordcount']['output'] = wordcount(tmp_text)
        log_stats.append(tmp_tokens)

        if exec_params['loglevel'] >= 1:
          print('*** Resonse: ***')
          print(textwrap.fill(tmp_text, 120) + '\n')
        print(str(tmp_tokens) + '\n')
        output_text_segments.append(tmp_text)
      else:
        tmp_text = '... Response from model ...'

      # Log prompts
      log_input = log_input + log_detail + '\n' + prompt_txt_final + '\n\n'
      log_input_SHORT = log_input_SHORT + log_detail + '\n' + prompt_txt_final_SHORT + '\n\n'
      log_input_output = log_input_output + log_detail + '\n' + prompt_txt_final + '\n\n' + tmp_text + '\n\n\n'

      # Log input / output comparison
      log_text = ''
      log_text = log_text + '| Step | ' + str(index + 1) + ' of ' + str(len(input_text_segments)) + ' |\n'
      log_text = log_text + '| -------- | ------- |\n'
      log_text = log_text + '| ' + str(wordcount(input_text)) + ' | ' + str(wordcount(tmp_text)) + ' |\n'
      log_text = log_text + '| ' + input_text.replace('\n','<br />') + ' | ' + tmp_text.replace('\n','<br />') + ' |\n'
      log_text = log_text + '\n'

      log_input_output_compare = log_input_output_compare + log_text

      # check if not all text segments/steps should be processed (eg. for debugging)
      if (index+1) == exec_params['stop-after-step']:
        break

  timer_end = time.time()
  timer_duration_secs = int(timer_end - timer_start)
  timer_duration_mins = int(timer_duration_secs/60*10)/10

  log_time_end = '\nEnded at: ' + datetime.now().strftime("%H:%M:%S") + '\n\nDuration: ' + str(timer_duration_mins) + ' Minutes\n'
  print(log_time_end)
  log_time = '\n\n\n***********************\n\n# Duration\n' + str(timer_duration_secs) + ' Seconds\n' + str(timer_duration_mins) + ' Minutes\n' + log_time_start + '\n' + log_time_end

  full_output_text = '\n\n'.join(output_text_segments)

  dot_pos = exec_params['result-filename'].rfind('.')
  input_text_filename_only1 = exec_params['result-filename'][:dot_pos]
  input_text_filename_only2 = exec_params['result-filename'][dot_pos:]
  output_filename_with_model = input_text_filename_only1 + '_' + exec_params['model'][:5] + input_text_filename_only2

  stats_aggregated = calculate_stats(log_stats)
  stats_text = '\n\n\n***********************\n\n# Statistics: \n'
  stats_text = stats_text + 'Wordcount ratio: ' + str(stats_aggregated['wordcount_ratio']) + '%, Costs: $' + str(stats_aggregated['prompt_tokens_price']) + '\n'
  stats_text = stats_text + '\n' + json.dumps(stats_aggregated, sort_keys=True, indent=2).replace('\n', '<br>') + '\n\n'
  #stats_text = stats_text + json.dumps(log_stats, sort_keys=True, indent=2) + '\n\n'
  print('Statistics:')
  print(stats_text)

  # save prompt log to disk
  save_and_backup(
      exec_params['proj-name'] + '/' + exec_params['folders']['text-output-logs'],
      output_filename_with_model + '.log.md',
      log_prompt + log_time + stats_text + log_input_output_compare + log_input_SHORT + log_input + log_input_output,
      True) # create .docx

  # save output only to disk (in log folder)
  save_and_backup(
      exec_params['proj-name'] + '/' + exec_params['folders']['text-output-logs'],
      output_filename_with_model + '.pure.md',
      full_output_text,
      False) # create .docx

  # save full output to disk
  save_and_backup(
      exec_params['proj-name'] + '/' + exec_params['folders']['text-output'],
      output_filename_with_model,
      full_output_text + '\n\n=== LOGS: ===\n\n\n\n' + log_prompt + log_time + stats_text + log_input_output_compare + log_input_SHORT + log_input + log_input_output,
      True) # create .docx

  # Always create .zip archive in root directory (for download)
  shutil.make_archive(exec_params['proj-name'], 'zip', exec_params['proj-name'])

  # return output path and filename (tuple)
  return exec_params['proj-name'] + '/' + exec_params['folders']['text-output'], output_filename_with_model



def execute_project(proj_name, model, prompt_lib, prompt_id, prompt_version, length_max_dyn, toc_text, stop_after_step, delete_output_dirs, folders, input_text_string, prompt_comment):
  create_test_project(proj_name, '')
  if delete_output_dirs == True:
    create_file_directory(proj_name + '/' + folders['text-output'], True)
    create_file_directory(proj_name + '/' + folders['text-output-logs'], True)
  # if input_text is contains '<File: >' do not create text file
  filename_regex = re.findall('(?sm)<File:>', input_text_string)
  if not filename_regex:
    # Save text to input dir
    f= open(proj_name + '/' + folders['text-input'] + '/input.txt','w+')
    f.write(input_text_string)
    f.close()
  else:
    # Check, if docx file is in input dir
    print('chechking for docx.')
    input_files_all = find_files(proj_name + '/' + folders['text-input'], [''], False)
    input_files_docx = find_files(proj_name + '/' + folders['text-input'], ['.docx'], False)
    if (len(input_files_all) == 1) and (len(input_files_docx) == 1):
      #convert docx to md
      dot_pos = input_files_docx[0].rfind('.')
      output_filename = input_files_docx[0][:dot_pos] + '.md'
      md_file = pypandoc.convert_file(input_files_docx[0], 'md', outputfile=output_filename)

  prompt_obj = get_prompt_from_lib(prompt_lib, prompt_id, model, prompt_version)
  #print(json.dumps(prompt_obj, sort_keys=False, indent=2))
  result_text = execute_prompt({
      'model': model,
      'prompt-name': prompt_id,
      'prompt-version': prompt_version,
      'prompt-comment': prompt_comment,
      'loglevel': 0,
      'stop-after-step': stop_after_step,
      'proj-name': proj_name,
      'input-filepath': '',
      'input-filename': '',
      'folders': folders,
      'prompt-obj': prompt_obj,
      'language': 'lang-same',
      'length-type': length_max_dyn,
      'length-max': 0.5,
      'toc-text': toc_text,
      'result-filename': 'response.md'
  })
  return result_text